[Reference](https://medium.com/@francescofranco_39234/dropout-regularization-with-keras-7b89651da252)

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.constraints import max_norm

In [2]:
# Model configuration
img_width, img_height         = 32, 32
batch_size                    = 250
no_epochs                     = 55
no_classes                    = 10
validation_split              = 0.2
verbosity                     = 1
max_norm_value                = 2.0

In [3]:
# Load CIFAR10 dataset
(input_train, target_train), (input_test, target_test) = cifar10.load_data()

# Reshape data based on channels first / channels last strategy.
# This is dependent on whether you use TF, Theano or CNTK as backend.
# Source: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
if K.image_data_format() == 'channels_first':
    input_train = input_train.reshape(input_train.shape[0],3, img_width, img_height)
    input_test = input_test.reshape(input_test.shape[0], 3, img_width, img_height)
    input_shape = (3, img_width, img_height)
else:
    input_train = input_train.reshape(input_train.shape[0], img_width, img_height, 3)
    input_test = input_test.reshape(input_test.shape[0], img_width, img_height, 3)
    input_shape = (img_width  , img_height, 3)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Convert target vectors to categorical targets
target_train = keras.utils.to_categorical(target_train, no_classes)
target_test = keras.utils.to_categorical(target_test, no_classes)

170498071/170498071 [==============================] - 2s 0us/step


In [4]:
# Create the model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), kernel_constraint=max_norm(max_norm_value), activation='relu', input_shape=input_shape, kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))
model.add(Conv2D(64, kernel_size=(3, 3), kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.50))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=max_norm(max_norm_value), kernel_initializer='he_uniform'))
model.add(Dense(no_classes, activation='softmax'))

In [5]:
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(input_train, target_train,
          batch_size=batch_size,
          epochs=no_epochs,
          verbose=verbosity,
          validation_split=validation_split
)

Epoch 1/55
160/160 [==============================] - 111s 677ms/step - loss: 2.0040 - accuracy: 0.2781 - val_loss: 1.7496 - val_accuracy: 0.3842
Epoch 2/55
160/160 [==============================] - 113s 707ms/step - loss: 1.5281 - accuracy: 0.4423 - val_loss: 1.4295 - val_accuracy: 0.5101
Epoch 3/55
160/160 [==============================] - 107s 669ms/step - loss: 1.3652 - accuracy: 0.5091 - val_loss: 1.2772 - val_accuracy: 0.5637
Epoch 4/55
160/160 [==============================] - 109s 681ms/step - loss: 1.2530 - accuracy: 0.5553 - val_loss: 1.1611 - val_accuracy: 0.6126
Epoch 5/55
160/160 [==============================] - 109s 679ms/step - loss: 1.1721 - accuracy: 0.5839 - val_loss: 1.0878 - val_accuracy: 0.6329
Epoch 6/55
160/160 [==============================] - 105s 655ms/step - loss: 1.1097 - accuracy: 0.6071 - val_loss: 1.0326 - val_accuracy: 0.6487
Epoch 7/55
160/160 [==============================] - 108s 674ms/step - loss: 1.0583 - accuracy: 0.6240 - val_loss: 0.9812 -

In [6]:
# Generate generalization metrics
score = model.evaluate(input_test, target_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.7763357758522034 / Test accuracy: 0.7472000122070312
